# <center>FACE RECOGNIZATION</center>

# Importing Libraries

In [6]:
import os
import cv2
import numpy as np
import scipy.linalg as s_linalg


# Class DataSet Protocol

In [7]:
class dataset:
    def __init__(self):
        # Diractory Name
        self.dir = ("C:\\Users\\PC\\Downloads\\1-(Tue&Wed)DAA109986\\Project\\FR\\Final\\Images" )
        
        #Image Hight And Width
        self.hight = 112
        self.width = 92
        
        #10: 7 train, 3 test
        
        #Image Size
        self.size = self.hight * self.width
        # List To Store Training Image
        self.StoreTrainImagesName = []
        # List To Store Person Number of All Images
        self.TrainIndex = []
        # List To Store How Many Images Of Each person
        self.CountEachpersonImage_Train = []
       
        # List To Store Test Image
        self.StoretestImagesName = []
        # List To Store Person Number of All Images Testing
        self.testIndex = []
        # List To Store Person Number of All Images
        self.CountEachpersonImage_test = []
        
        # List To Store Subject Name For Ecuilidien Distance
        self.IndexTarget= []
        
        # Matrix To Store the Vector Calling In GetImageVector() Function
        self.ImgVector_matrix = np.zeros((10304,280))                   # Size  10304 x 280
        
        # mat = np.zeros((2,2))
        # mat = [0,0
        #         0,0]
            
    def ProtoCol(self,required_no):
        per_no = 0
        
        # For get Folder Name In to Directory
        for name in os.listdir(self.dir):
            dir_path = os.path.join(self.dir, name)
            
            # self.dir = ("D:\\KIET\\4thSem\\DMTheory\\Project\\Help\\PROJECT\\PROJECT\\ORL" )
            #name = S1, S2, S3 .. , s40
            # dir_path = "D:\\KIET\\4thSem\\DMTheory\\Project\\Help\\PROJECT\\PROJECT\\ORL\\S41"
            
            #Condition is Directory is Available
            if os.path.isdir(dir_path):
                if len(os.listdir(dir_path)) >= required_no:
                    i = 0
                    # For get Image Name In to Folder
                    for img_name in os.listdir(dir_path):
                        img_path = os.path.join(dir_path, img_name)

                        # dir_path = "D:\\KIET\\4thSem\\DMTheory\\Project\\Help\\PROJECT\\PROJECT\\ORL\\S1"
                        # img_name = 1.pgm, 2.pgm .. 7.pgm
                        # img_path = "D:\\KIET\\4thSem\\DMTheory\\Project\\Help\\PROJECT\\PROJECT\\ORL\\S1\\1.pgm"
                        
                        #training 
                        # i = 0 to 6 = 7
                        if i < required_no: 
                            
                            self.StoreTrainImagesName.append(img_path)  
                            self.TrainIndex.append(per_no)  
                            
                            if len(self.CountEachpersonImage_Train) > per_no: #(7) > 0 => 
                                self.CountEachpersonImage_Train[per_no] += 1
                            else:
                                self.CountEachpersonImage_Train.append(1)  
                            
                            if i == 0:
                                self.IndexTarget.append(name) 
                                # [s1,s2,s3...,s40]
                             
                        #testing
                        # i = 7 to 9 = 7 ,8, 9
                        else:
                            self.StoretestImagesName.append(img_path)  
                            self.testIndex.append(per_no)   
                            
                            if len(self.CountEachpersonImage_test) > per_no:
                                self.CountEachpersonImage_test[per_no] += 1
                            else:
                                self.CountEachpersonImage_test.append(1)  
                        i += 1
                    per_no += 1
    
    def GetImageVector(self):
        i = 0
        for name in self.StoreTrainImagesName:
            # For Read Image
            gray = cv2.imread(name, 0)
            # For resize Image
            gray = cv2.resize(gray, (self.hight, self.width))
            # Store pixels In matrix (112 , 92)
            mat = np.asmatrix(gray)
            # Convert into Vector and Save respective Column
            self.ImgVector_matrix[:, i] = mat.ravel()
            i += 1
        return self.ImgVector_matrix


# Principal Component Anylasis PCA

In [8]:
class Principal_Component_Anylasis:
    def __init__(self, VectorImage,TrainImages , TrainIndex , TargetIndex , CountTraining_img): 
        self.VectorImage = VectorImage
        self.CountTraining_img = CountTraining_img
        self.TrainImages = TrainImages
        self.TrainIndex = TrainIndex
        self.TargetIndex = TargetIndex
        self.lengthImages = len(TrainImages)
        self.size = 112*92
        self.Mean_Matrix = np.zeros(self.size)
        self.MeanDiff_Matrix = np.zeros((self.size,self.lengthImages)) #10304 x 280
        self.Eigan_Space = np.zeros((self.size,self.lengthImages))     #10304 x 280
        
    def getMean(self):
        Mean = np.mean(self.VectorImage,1)
        #Transpose of Means
        self.Mean_Matrix = np.matrix(Mean).T
        # Mean Difference For Each image
        self.VectorImage = self.VectorImage - self.Mean_Matrix
        print(self.VectorImage)
        print(self.VectorImage.shape)
    
    
    def Give_Best_Values(self, EigenValues):
        Sum = np.sum(EigenValues)
        features = 90
        Best = features * Sum/100
        temp = 0
        BestValue = 0
        while temp < Best:
            temp += EigenValues[BestValue]
            BestValue += 1 
        return BestValue

    def GiveEigan(self):
        Eiganmatrix ,EigenValues ,VectorT=  s_linalg.svd(self.VectorImage, full_matrices=True)
        EiganvectorMatrix = np.matrix(Eiganmatrix)
        bestValues = self.Give_Best_Values(EigenValues)
        self.EigenVector = Eiganmatrix[:, 0:bestValues]
        return self.EigenVector
    
    def EigenSpace(self):
        EigenVector = self.GiveEigan()
        self.Eigan_Space = np.dot(EigenVector.T, self.VectorImage)
        return self.Eigan_Space.T
    
    def For_textImages(self, test_Image):
        img_vec = np.asmatrix(test_Image).ravel()
        img_vec = img_vec.T
        # Mean of Single Image 
        _Mean= ((self.Mean_Matrix * len(self.TrainIndex)) + img_vec) / (len(self.TrainIndex) + 1)
        # Vector Of Single Image
        img_vec = img_vec - _Mean
        EigenVector = self.GiveEigan()
        EiganSpace = np.dot(EigenVector.T, img_vec)
        return EiganSpace
        
    def Ecluidien_Distance(self, Eigan_Space):
        classes = len(self.CountTraining_img) # => 40 
        start = 0
        EcluidienDistance = []
        for i in range(classes):            
            Mapping = self.Eigan_Space[:, int(start): int(start + self.CountTraining_img[i])]
            mean_temp = np.mean(Mapping, 1)
            start = start + self.CountTraining_img[i]
            dist = np.linalg.norm(Eigan_Space - mean_temp)
            EcluidienDistance.append(dist)   
        _min = np.argmin(EcluidienDistance) 
        return self.TargetIndex[_min]      

# Input Outputs

In [9]:
requres_images = 7
dataset = dataset()

trainImage = dataset.StoreTrainImagesName
trainingLabel = dataset.TrainIndex
No_of_TrainImage = dataset.CountEachpersonImage_Train


testImage = dataset.StoretestImagesName
testingLabel = dataset.testIndex
No_of_testImage = dataset.CountEachpersonImage_test

imageTraget = dataset.IndexTarget

protocol = dataset.ProtoCol(requres_images)

ImageMatrix_to_Vector = dataset.GetImageVector()



# PCA Object and Loop

In [10]:

pca = Principal_Component_Anylasis(ImageMatrix_to_Vector,trainImage,trainingLabel,imageTraget,No_of_TrainImage)
EiganVector = pca.GiveEigan()
hight = 112
width = 92
Correct = 0
wrong = 0
count = 0

In [ ]:
for test_Image in testImage:
    if not os.path.isfile(test_Image):
        continue
    gray = cv2.imread(test_Image, 0)
    gray = cv2.resize(gray, (hight, width))
    
    #__For DisPlaying Imge__
    Reshaping = np.reshape(gray,[width,hight])
    ImgArray = np.array(Reshaping,dtype=np.uint8)
    ShowRecize = cv2.resize(ImgArray,(500,500))
        
    ForText = pca.For_textImages(gray)
    
    FindSubjectName = pca.Ecluidien_Distance(ForText)
    Target_Index = testingLabel[count]

    SubjectName = imageTraget[Target_Index]

    if FindSubjectName is SubjectName:
        Correct +=1
        print(f"Matched : Subject is : {FindSubjectName}")
    else:   
        wrong +=1
        print(f"Not Matched : Subject is : {FindSubjectName}") 
    
    count += 1
    
print(f"Total Number Of Matched Image Is : {Correct}")
print(f"Total Number Of Not Matched Image Is : {wrong}")

Matched : Subject is : p1
Matched : Subject is : p1
Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject is : p1
Not Matched : Subject 

# THANK YOU

In [ ]:
# Mean Difference: Zero mean, it's movement is zero.
# calculate varience of dataset (mean zero)
# varience:
# (Generates a new pattren)
# (tells connection between data either it is directly propotional or inversly propostional through + - sign)
# (sign has more worth than magnitude)
# Through variance we find EigenSpace which is based on Eigen Value and Eigen Vector
# Eigen Value and Eigen Vector help us in selecting dimensions
# we map data of (mean difference) on Eigen vector's dimensions
#
